In [ ]:
#install the Ollama tool on a Linux system, runs the Ollama server in a separate thread, and downloads the "gemma2" model while 
# listing available models.

!curl -fsSL https://ollama.com/install.sh | sh
import subprocess
process = subprocess.Popen("ollama serve", shell=True) #runs on a different thread
#Download model
!ollama pull gemma2
!pip install ollama
import ollama
!ollama list
!ollama pull gemma2
!ollama list

In [ ]:
# response = ollama.chat(model='gemma2', messages=[
#   {
#     'role': 'user',
#     'content': f"""Convert the following unit abbreviations to their full unit names while keeping the numerical value unchanged. Here is the entity-unit map to be used:
# width, depth, height: 'cm' to 'centimetre', 'ft' to 'foot', 'in' to 'inch', 'm' to 'metre', 'mm' to 'millimetre', 'yd' to 'yard'
# item_weight, maximum_weight_recommendation: 'g' to 'gram', 'kg' to 'kilogram', 'µg' to 'microgram', 'mg' to 'milligram', 'oz' to 'ounce', 'lb' to 'pound', 't' to 'ton'
# voltage: 'kV' to 'kilovolt', 'mV' to 'millivolt', 'V' to 'volt'
# wattage: 'kW' to 'kilowatt', 'W' to 'watt'
# item_volume: 'cL' to 'centilitre', 'cu ft' to 'cubic foot', 'cu in' to 'cubic inch', 'cup', 'dL' to 'decilitre', 'fl oz' to 'fluid ounce', 'gal' to 'gallon', 'imp gal' to 'imperial gallon', 'L' to 'litre', 'µL' to 'microlitre', 'mL' to 'millilitre', 'pt' to 'pint', 'qt' to 'quart'
# For example, convert '34cm' to '34 centimetre' or '12ft' to '12 foot'. Apply these transformations to any given input."""
#   },
# ])
# print(response['message']['content'])

In [ ]:
import os

# This function retrieves the full file paths of all CSV files in a specified folder and its subfolders by traversing the directory structure.
def get_csv_file_paths(folder_path):
    csv_file_paths = []
    
    # Loop through all files in the folder
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Check if the file has a '.csv' extension
            if file.endswith('.csv'):
                # Add the full file path to the list
                csv_file_paths.append(os.path.join(root, file))
    
    return csv_file_paths

In [ ]:
import re
import string

# This function cleans a given text by converting it to lowercase, removing HTML tags, 
# new line characters, and extra spaces to produce a standardized string.
def clean(text):
    text = text.lower() #Converting to lowerCase
    text = re.sub('<.*?>+',' ',text) #removing HTML Tags
    text = re.sub('\n', ' ',text) #removal of new line characters
    text = re.sub(r'\s+', ' ',text) #removal of multiple spaces
    return text

# PUT FOLDER NAME

In [ ]:
import pandas as pd
folder_name = "put_your_folder_name" #Put the folder name where files are located of the files (from Mini-CPM)

In [ ]:
import time


# loads CSV files from a specified folder, processes each row to send prediction values to the Ollama chat model, 
# collects the responses, and saves the updated predictions to a new CSV file.

folder_path = f"/kaggle/input/{folder_name}"
files = get_csv_file_paths(folder_path)

results=[]
for file in files:
    df = pd.read_csv(file)
    for index, row in df.iterrows():
        start = time.time()
        value = row['prediction']  # Get the prediction value

        # Send the prediction to the ollama.chat model
        response = ollama.chat(
            model='gemma2',
            messages=[
                {
                    'role': 'user',
                    'content': f"""{value}. Only return number and full form of unit, nothing else.""",
                }
            ]
        )

        # Replace the prediction value with the response content
        results.append({'index':row['index'],'prediction':clean(response['message']['content'])})
        end = time.time()
        logs = {
            'index': row['index'],
            'prediction': response,
            'value':value,
            'time': end-start
        }

        print(f"Done: {logs}")

# Save the updated DataFrame back to CSV
df1 = pd.DataFrame(results)

# Save DataFrame to CSV
df1.to_csv(f'output_{folder_name}.csv',index=False)

print("CSV updated successfully.")